In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Toys_and_Games.json.gz

--2021-01-26 04:24:54--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Toys_and_Games.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 799541605 (763M) [application/octet-stream]
Saving to: ‘meta_Toys_and_Games.json.gz’

meta_Toys_and_Games 100%[===================>] 762.50M  42.5MB/s    in 18s     

2021-01-26 04:25:13 (41.3 MB/s) - ‘meta_Toys_and_Games.json.gz’ saved [799541605/799541605]



In [ ]:
### load the meta data
 
data = []
with gzip.open('meta_Toys_and_Games.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))
 
# first row of the list
print(data[0])

633883
{'category': ['Toys & Games', 'Puzzles', 'Jigsaw Puzzles'], 'tech1': '', 'description': ["Three Dr. Suess' Puzzles: Green Eggs and Ham, Favorite Friends, and One Fish Two Fish Red Fish Blue Fish"], 'fit': '', 'title': 'Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle', 'also_buy': [], 'image': ['https://images-na.ssl-images-amazon.com/images/I/51rn8TxbcoL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/51j1Fep1niL._SS40_.jpg'], 'tech2': '', 'brand': 'Dr. Seuss', 'feature': ['Three giant floor puzzles', 'Includes: Dr. Suess Green Eggs and Ham, Favorite Friends, and One Fish Two Fish Blue Fish', 'Each puzzle has 48 pieces', 'Ages 3 and up'], 'rank': ['>#2,230,717 in Toys & Games (See Top 100 in Toys & Games)', '>#57,419 in Toys & Games > Puzzles > Jigsaw Puzzles'], 'also_view': [], 'main_cat': 'Toys & Games', 'similar_item': '', 'date': '', 'price': '', 'asin': '0000191639'}


In [ ]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

633883


In [ ]:
df.head(2)

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Toys & Games, Puzzles, Jigsaw Puzzles]",,"[Three Dr. Suess' Puzzles: Green Eggs and Ham,...",,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle,[],[https://images-na.ssl-images-amazon.com/image...,,Dr. Seuss,"[Three giant floor puzzles, Includes: Dr. Sues...","[>#2,230,717 in Toys & Games (See Top 100 in T...",[],Toys & Games,,,,0000191639,NaN
1,[],,[<b>Prepare to be Afraid!</b><br /><br />The B...,,Pathfinder: Book of Beasts - Legendary Foes,[],[],,Pathfinder Roleplaying Jon Brazer Productions,[],"[>#2,294,535 in Toys & Games (See Top 100 in T...",[],Toys & Games,,,.a-box-inner{background-color:#fff}#alohaBuyBo...,0004950763,NaN


In [ ]:
df.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'image',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'details'],
      dtype='object')

### Take the columns "**main_cat**","**asin**","**title**".
**WHY ??**


*   "main_cat" : main category (to filter out only Toys & Games)
*   "asin": unique product id (to merge metadata dataframe to product ratings dataframe)
*   "title": title of product 

In [ ]:
df2=df[['main_cat','asin','title']]

In [ ]:
df2.head()

,main_cat,asin,title
0,Toys & Games,0000191639,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle
1,Toys & Games,0004950763,Pathfinder: Book of Beasts - Legendary Foes
2,Toys & Games,0005069491,Nursery Rhymes Felt Book
3,Toys & Games,0004983289,Dutch Blitz Card Game
4,Toys & Games,0006466222,Magic Pen Painting (Marvel Super Heroes)


# Data Cleaning

**1**. **main_cat**

In [ ]:
df2['main_cat']

0                     Toys & Games
1                     Toys & Games
2                     Toys & Games
3                     Toys & Games
4                     Toys & Games
                    ...           
633878    Tools & Home Improvement
633879                Toys & Games
633880                Toys & Games
633881                Toys & Games
633882                        Baby
Name: main_cat, Length: 633883, dtype: object

**OBSERVATIONS**


*   There are other category products also there in this dataset 
*   we need to remove other category products  like Movies & TV, Books etc


In [ ]:
#Lets check which category has how many products
df2['main_cat'].value_counts()

Toys & Games                                                                                                                                                                                      574070
Amazon Home                                                                                                                                                                                        12856
Baby                                                                                                                                                                                                9524
Sports & Outdoors                                                                                                                                                                                   6112
<img src="https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-store-new._CB520838675_.png" class="nav-categ-image" alt="AMAZON FASHION"/>                            

In [ ]:
# take cell phones and accessories 
df3 = df2[df2['main_cat'] == 'Toys & Games']

In [ ]:
df3['main_cat'].value_counts()

Toys & Games    574070
Name: main_cat, dtype: int64

In [ ]:
df3.head(3)

,main_cat,asin,title
0,Toys & Games,0000191639,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle
1,Toys & Games,0004950763,Pathfinder: Book of Beasts - Legendary Foes
2,Toys & Games,0005069491,Nursery Rhymes Felt Book


**2.asin**

In [ ]:
df3['asin']

0         0000191639
1         0004950763
2         0005069491
3         0004983289
4         0006466222
             ...    
633876    B01HJBVP7E
633877    B01HJC53OE
633879    B01HJDGVFS
633880    B01HJDUNRU
633881    B01HJFAGJI
Name: asin, Length: 574070, dtype: object

In [ ]:
df3['asin'].isnull().sum()

0

**OBSERVATIONS**

There may be chance of null values in this column bacause the price column shows zero null values but in that column there are null values exist

**String.isalnum()** function returns **True** if the string contains only alphanumeric characters and **False** if not.

Reference
[link text](https://pythonexamples.org/python-string-check-if-alphanumeric/)

In [ ]:
# Examples
str = '6786501710' # asin startswith number
result1 = str.isalnum() 
print(result1)

str2 = 'B01HJH76J4' # asin startswith alphabet
result2 = str2.isalnum() == True
print(result2)

str3 = ""
result3 = str3.isalnum()
print(result3)

True
True
False


In [ ]:
#Function to remove null values from asin column
def remove_null_in_asin(x):
  if x.isalnum() == True:
    return x
  else:
    return None

In [ ]:
df4 = df3.copy()
df4['asin'] = df4['asin'].apply(remove_null_in_asin)

In [ ]:
df4['asin'].isnull().sum()

0

**OBSERVATIONS**

No null values in asin column

In [ ]:
#Check whether do we have duplicate asin ?
df4['asin'].value_counts()

B00005QEFQ    2
B00020UWJC    2
B00009VE3P    2
B000059EHQ    2
B00009IMCW    2
             ..
B00G4FBXEW    1
B000UO5S3U    1
B00ZF7G7GO    1
B0081GGOOG    1
B00250Y8OC    1
Name: asin, Length: 565050, dtype: int64

**OBSERVATIONS**

*   There are repeated asin  or duplicate asin 

In [ ]:
len(df4['asin'])

574070

In [ ]:
#remove the duplicate products
df4.drop_duplicates(subset="asin",keep="first",inplace=True)

In [ ]:
df4['asin'].value_counts()

B00BWYFTNG    1
B00IX2M68Q    1
B00C8TMHOS    1
B0171TS298    1
B00DFHZ0BE    1
             ..
B00H2C1IM8    1
B006JQUEH8    1
B000O53HZQ    1
B01H0MVHNO    1
B00250Y8OC    1
Name: asin, Length: 565050, dtype: int64

In [ ]:
len(df4['asin']) #574070 to 565050

565050

**3.title**

In [ ]:

df4['title']

0            Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle
1               Pathfinder: Book of Beasts - Legendary Foes
2                                  Nursery Rhymes Felt Book
3                                     Dutch Blitz Card Game
4                  Magic Pen Painting (Marvel Super Heroes)
                                ...                        
633876    jrelecs 2PCS 4MM Banana Plug to EC5 Male Conve...
633877    Marvel Funko Pop Black Suit Spider-Man #79 (Gl...
633879    Micord Baby Float Toddler Swimming Inflatable ...
633880    Premium Swimming Pool Float Hammock, Inflatabl...
633881    Lewo Wooden Baby Toddler Toys Circle First Bea...
Name: title, Length: 565050, dtype: object

The strip() method removes any leading (spaces at the beginning) and trailing (spaces at the end) characters

In [ ]:
#lets remove start and end whitespaces in each title
df4['title'] = df4['title'].str.strip()

In [ ]:
#Check whether do we have duplicate title or repeated title ?
df3['title'].value_counts()

Young Choi's Kids Toy Single Large Bubble Gun Shooter for Boys and Girls, Toy Bubble Blaster with LED Flashing Lights and Sounds Include Extra Refill Bottle, Outdoors Toy (Batteries Include)                  162
Magnetic Gun Mount &amp; Holder for Car and Vehicle - 43 Lbs Rated - Heavy Duty Concealed Rubber Coated Magnet Gun Safe Holster for Pistol Handgun Revolver Shotgun Rifle Desk Truck Home - Firearm Accessor     66
Water Drawing Mat &amp; 2 Magic Pens Medium 14.6x18.2&rdquo;. Arts Crafts Doodle Mat . No Mess Coloring. Play Mat for Kids. Water Painting Magic Mat Pad . Creative Learning                                     51
Vet My Pet Dogs Cats - 1500mg - Natural Support Hip &amp; Joint - Provides Anxiety Relief - Better Mood Sleep - 100% Organic Treat Food Supplement Grown Extracted in USA All Pets                               35
 Magic                                                                                                                                                  

**OBSERVATIONS**
*   there are repeated titles in dataset

In [ ]:
len(df4['title'])

565050

In [ ]:
#remove the duplicate titles
df4.drop_duplicates(subset="title",keep="first",inplace=True)

In [ ]:
df4['title'].value_counts()

Kole Police Play Plastic Handcuffs                                               1
Plants Vs Zombies Plush Toy Peashooter 17cm/6.7&quot; Tall (Small Size)          1
Deoxys Ex Plasma Freeze 53/116 Pokemon Card Rare                                 1
Fascinations Metal Earth Transformers Bumblebee 3D Metal Model Kit               1
Magic the Gathering MTG - 10 Magic 2015 M15 Assorted Rare Cards                  1
                                                                                ..
Grey Falcon - Audubon Plush Bird (Authentic Bird Sound)                          1
Citadel Scorched Grass                                                           1
Yu-Gi-Oh! - Wattcancel (PHSW-EN073) - Photon Shockwave - 1st Edition - Common    1
Animal Hospital Play Vet Set                                                     1
                                                                                 1
Name: title, Length: 559999, dtype: int64

In [ ]:
len(df4['title']) #565050 to 559999

559999

In [ ]:
df4['title'].isnull().sum()

0

**OBSERVATIONS**

There may be chance of null values,blank values in this column bacause the price column shows zero null values but in that column there are null values exist

You Cannot Use Python Regex in startswith() [link text](https://https://blog.finxter.com/regex-startswith-python/)

So using tuple in startswith() function

In [ ]:
alphanumeric = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
                '1','2','3','4','5','6','7','8','9','0']
def remove_null_in_title(x):
  if x.startswith(tuple(alphanumeric)):
    return x
  else:
    return None

In [ ]:
df5 = df4.copy()
df5['title'] = df5['title'].apply(remove_null_in_title)

In [ ]:
df5['title'].isnull().sum()

2735

In [ ]:
#drop the null values
df6 = df5.dropna()

In [ ]:
df6.shape

(557264, 3)

In [ ]:
df6.head(2)

,main_cat,asin,title
0,Toys & Games,0000191639,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle
1,Toys & Games,0004950763,Pathfinder: Book of Beasts - Legendary Foes


In [ ]:
df6.isnull().sum()

main_cat    0
asin        0
title       0
dtype: int64

In [ ]:
#drop the column 'main_cat' column
df6 = df6.drop(['main_cat'],axis=1)

In [ ]:
df6.shape

(557264, 2)

In [ ]:
#convert dataframe to csv
df6.to_csv("Toys and Games Metadata dataset.csv",index=False)